In [4]:
import json
import os
import numpy as np

In [5]:
def read_json(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

# Global Variables

In [15]:
data_dir = "/home/amogh/spatial_guide_intern/data/AR_Recorder"
object_name = "elephant_terracota"
object_id = 102
output_dir = f"/home/amogh/research/datasets/lm_format/train/"

In [7]:
object_dir = os.path.join(data_dir, object_name)
json_file_name = [file_name for file_name in os.listdir(object_dir) if file_name.split(".")[-1] == 'json' ][0]
json_file_path = os.path.join(object_dir, json_file_name)

In [9]:
data = read_json(json_file_path)

In [12]:
data_keys = list(data.keys())

cam_intrinsics = {}
dict1 = {}
for i in range(len(data_keys)):
    dict1["cam_K"] = list(np.array(data[data_keys[i]]['cameraIntrinsics']).reshape(-1,))
    dict1["depth_scale"] = 0.1
    cam_intrinsics[str(i)] = dict1

cam_intrinsics_path = output_dir + f"{object_id:06d}/scene_camera.json"
# with open(cam_intrinsics_path, "w") as f:
#     json.dump(cam_intrinsics, f, indent=4)

# Extrinsics

In [21]:
extrinsics_dict = {}
for i in range(len(data_keys)):

    extrinsics = np.array(data[data_keys[i]]['cameraTransform'])
    scale_factor = extrinsics[-1,-1]
    R = extrinsics[:3,:3] / scale_factor
    t = extrinsics[:3,-1] / scale_factor

    frame_list = []
    frame_dict = {}
    frame_dict['cam_R_m2c'] = list(R.reshape(-1,))
    frame_dict['cam_t_m2c'] = list(t)
    frame_dict['obj_id'] = object_id
    frame_list.append(frame_dict)
    extrinsics_dict[str(i)] = frame_list

extrinsics_json_path = output_dir + f"{object_id:06d}/scene_gt.json"

# with open(extrinsics_json_path, "w") as f:
#     json.dump(extrinsics_dict, f, indent=4)